1. what is the approximate depth of a Decision Tree trained(without restrictions) on a traing set with 1 million instance?

2. Is a node's Gini impurity generally lower or greater than its parent's? Is it generaaly lover/greater, or always lower/greater?

3. If a Decision Treee is overfitting the training set, is it a good idea to try decreasing max_depth?

If a Decision tree is overfitting the training set, it may be a good idea to decrease max_depth, since this will constrain the model, regularizing it.

4. If a Decision Tree is overfitting the training set, is it a good idea to try scaling the input features?


Decision tree donot care whether or not the training set is scaled or centered; that's one of the nice things about them. so if a decision tree underfits the training set, scaling the input features will just be a waste of time.

5. If it takes one hour to train a Decision tree on a training set containing 1 million instance, roughly how much time will it take to train another Decision tree on a training set containing 10 million instances?

6. If you training set contains 100,000 instances, will setting presort = True speed up training?

7. Train and fine-tune a Decision Tree for the moon dataset.

In [2]:
from sklearn.datasets import make_moons
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV

In [3]:
X,y = make_moons(n_samples=10000, noise=0.4, random_state=42)

In [5]:
X.shape

(10000, 2)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [9]:
from sklearn.tree import DecisionTreeClassifier

In [11]:
params = {'max_leaf_nodes':list(range(2,100)), 'min_samples_split':[2,3,4]}
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1, verbose=1, cv=3)
grid_search_cv.fit(X_train,y_train)

Fitting 3 folds for each of 294 candidates, totalling 882 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 234 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 882 out of 882 | elapsed:    4.7s finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99], 'min_samples_split': [2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       s

In [12]:
grid_search_cv.best_estimator_

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=17,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [13]:
grid_search_cv.best_score_

0.8555

In [14]:
from sklearn.metrics import accuracy_score

y_pred = grid_search_cv.best_estimator_.predict(X_test)
accuracy_score(y_test, y_pred)

0.8695

In [15]:
y_pred2 = grid_search_cv.predict(X_test)
accuracy_score(y_test, y_pred2)

0.8695

8. Grow a forest